In [14]:
import pandas as pd
import numpy as np
import re, unicodedata

# rename columns

In [4]:
df = pd.read_csv("raw_dataset.csv")

In [13]:
# Example: rename columns manually
df.rename(columns={
    "Fish ID": "fish_id",
    "Species": "species",
    "CommonName": "common_name",
    "Lake Name": "waterbody_name",
    "Kingdom": "kingdom",
    "Phylum": "phylum",
    "Class": "class",
    "Order": "order",
    "Family": "family",
    "Genus": "genus",
    "Status": "status",
    "FeedingType": "feeding_type",
    "Tmax": "temp_max",
    "Wmax": "weight_max",
    "Lmax": "length_max",
    "TempPref_min": "temp_pref_min",
    "TempPref_max": "temp_pref_max",
    "TempRange": "temp_range",
    "MeanFecundity": "fecundity_mean",
    "MinFecundity": "fecundity_min",
    "MaxFecundity": "fecundity_max",
    "Trophic Level Estimate": "trophic_lvl_estimate",
    "Trophic Level": "trophic_lvl",
    "Lake_pH": "wb_ph",
    "Lake_Salinity": "wb_salinity",
    "Lake_Dissolved Oxygen (mg/L)": "wb_do",
    "Lake_Biochemical Oxygen Demand (mg/L)": "wb_bod",
    "Lake_Turbidity (NTU)": "wb_turbidity",
    "Lake_Temperature": "wb_temp",
}, inplace=True)

# drop sources column and (extra) empty column
# .drop(["Sources (INSERT LINK)", "Unnamed: 30"], axis=1, inplace=True)

print(df.columns)


Index(['fish_id', 'species', 'common_name', 'waterbody_name', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'status', 'feeding_type',
       'temp_max', 'weight_max', 'length_max', 'temp_pref_min',
       'temp_pref_max', 'temp_range', 'fecundity_mean', 'fecundity_min',
       'fecundity_max', 'trophic_lvl_estimate', 'trophic_lvl', 'wb_ph',
       'wb_salinity', 'wb_do', 'wb_bod', 'wb_turbidity', 'wb_temp'],
      dtype='object')


# Split mix max values of water params

In [15]:
# columns you said contain ranges
range_cols = [
    "wb_ph", "wb_salinity", "wb_do", 
    "wb_bod", "wb_turbidity", "wb_temp"
]

def normalize_dashes(text):
    if pd.isna(text):
        return text
    # Replace various Unicode dashes with a normal hyphen
    return re.sub(r"[–—-–]", "-", str(text))

def normalize_text(s):
    """Normalize unicode dashes/spaces and strip extraneous chars."""
    if pd.isna(s):
        return s
    s = str(s)
    s = unicodedata.normalize("NFKC", s)              # normalize weird unicode
    # replace various dash/minus characters with a normal hyphen
    for ch in ("\u2013", "\u2014", "\u2212", "–", "—"):
        s = s.replace(ch, "-")
    s = s.replace(" to ", "-")     # '10 to 15' -> '10-15'
    s = s.replace("–", "-")
    # remove surrounding punctuation that may be trailing/leading
    s = s.strip().strip(",; ")
    return s

def split_range_cell(s, strategy="first"):
    """
    Parse a cell s and return (min, max) as floats or (None, None).
    strategy:
        - "first": take the first numeric range found (default)
        - "span": take min of all lows and max of all highs (cover span)
        - "mean": take mean of lows and mean of highs
    """
    s2 = normalize_text(s)
    if pd.isna(s2) or s2 == "":
        return (None, None)
    
    # find explicit ranges like "10-15" (with optional decimals and negative)
    range_matches = re.findall(r'(-?\d+(?:\.\d+)?)\s*-\s*(-?\d+(?:\.\d+)?)', s2)
    if range_matches:
        pairs = [(float(a), float(b)) for a, b in range_matches]
        if strategy == "first":
            return pairs[0]
        elif strategy == "span":
            lows = [a for a, b in pairs]
            highs = [b for a, b in pairs]
            return (min(lows), max(highs))
        elif strategy == "mean":
            lows = [a for a, b in pairs]
            highs = [b for a, b in pairs]
            return (sum(lows)/len(lows), sum(highs)/len(highs))


    # if no explicit ranges, try to extract numbers in the string
    nums = re.findall(r'(-?\d+(?:\.\d+)?)', s2)
    if nums:
        numsf = [float(x) for x in nums]
        if len(numsf) == 1:
            return (numsf[0], numsf[0])        # single number -> min==max
        else:
            # multiple numbers but no dash: assume pairings left-to-right
            return (numsf[0], numsf[1])
    
    # nothing found
    return (None, None)

# APPLY to dataframe, producing _min and _max for each range column
for col in range_cols:
    if col in df.columns:
        # if column already numeric, just copy to min/max
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col + "_min"] = df[col].astype(float)
            df[col + "_max"] = df[col].astype(float)
        else:
            # choose strategy: "first" is safest when there are multiple ranges in a cell
            df[[col + "_min", col + "_max"]] = df[col].apply(lambda x: pd.Series(split_range_cell(x, strategy="first")))
    else:
        print(f"Warning: column '{col}' not found in df")
        

for col in range_cols:
    min_col = col + "_min"
    max_col = col + "_max"
    
    if min_col in df.columns and max_col in df.columns:
        # Swap where min > max
        swap_mask = df[min_col] > df[max_col]
        df.loc[swap_mask, [min_col, max_col]] = df.loc[swap_mask, [max_col, min_col]].values


# Replace obvious non-numeric placeholders with NaN
placeholders = ['nr', '-', '\x96', '\x97']
for col in range_cols:
    if col in df.columns:
        df[col] = df[col].replace(placeholders, pd.NA)

# Quick diagnostics: how many missing in each new min/max
print("\nMissing counts after splitting:")
for col in range_cols:
    if col in df.columns:
        print(col, "min NA:", df[col + "_min"].isna().sum(), "max NA:", df[col + "_max"].isna().sum())

# Show sample rows where a particular column failed to parse
example_col = "WB_pH"
if example_col in df.columns:
    mask = df[example_col + "_min"].isna() & df[example_col].notna()
    if mask.any():
        print(f"\nSample problematic '{example_col}' cells (original -> parsed):")
        display(df.loc[mask, [example_col]].head(20))


Missing counts after splitting:
wb_ph min NA: 747 max NA: 747
wb_salinity min NA: 1144 max NA: 1144
wb_do min NA: 1161 max NA: 1161
wb_bod min NA: 3384 max NA: 3384
wb_turbidity min NA: 2680 max NA: 2680
wb_temp min NA: 837 max NA: 837


In [19]:
df.to_csv("min_max_split.csv", index=False)

# Fix waterbody names

In [20]:
pd.set_option("display.max_colwidth", None)

In [120]:
pattern = r"[^A-Za-z\s,áéíóúñüäýãôõâÉè()/&-.']"

df_symbols = df[df['waterbody_name'].str.contains(pattern, regex=True, na=False)]

print(df_symbols[['waterbody_name']].drop_duplicates().head(50))

df['waterbody_name'] = df['waterbody_name'].replace({
    "Ganga River — Bijnor": "Ganga River, Bijnor",
    "Ganga River — Narora": "Ganga River, Narora",
    "Ganga River — Farrukhabad": "Ganga River, Farrukhabad",
    "Ganga River — Prayagraj (Allahabad)": "Ganga River, Prayagraj",
    "Ganga River — Varanasi / Prayagraj": "Ganga River, Varanasi",
    "Padma River — Bheramara (Kushtia)": "Padma River, Bheramara Kushtia",
    "Brahmaputra River — Guwahati / Kamrup stretches": "Brahmaputra River, Guwahati",
    "Langueyú stream — Urban reach": "Langueyú stream, Urban reach",
    "Garças Lagoon": "Garcas Lagoon",
    "Shari–Goyain / Shari Goyain River (Bangladesh)": "Shari Goyain River, Bangladesh",
    "Nuʻuanu Reservoir (Oʻahu, Hawaii)": "Nuʻuanu Reservoir, Oʻahu, Hawaii",
    "Chilika Lake — Rambha": "Chilika Lake, Rambha",
    "Lake Worth Drainage Canal L-14": "Lake Worth Drainage Canal L-14",
    "Ganges River (selected reaches; e.g., Haridwar/Allahabad studies)": "Ganges River",
    "Lake Eber (Akarçay basin)": "Lake Eber, Akarcay basin",
    "Lake Beyşehir": "Lake Beysehir",
    "Lake Akşehir": "Lake Aksehir",
    "Mekong Delta — freshwater tidal zone (Vietnam)": "Mekong Delta, Vietnam",
    "Mekong Delta — brackish estuaries (Vietnam)": "Mekong Delta, Vietnam",
    "Moreton Bay — mangrove creeks (tidal)": "Moreton Bay",
    "Jubail / northern Arabian Gulf (Saudi coast) — coastal waters where specimen recorded": "Jubail Northern Arabian Gulf",
    "McNutt Creek (tributary, Athens / Oconee drainage) — wild records": "McNutt Creek",
    "Chalakudy River , Puliyarkutti 8th bridge": "Chalakudy River, Puliyarkutti 8th bridge",
    "Rupnarayan River (Purba Medinipur, West Bengal) — sampling stations A/B/C": "Rupnarayan River, Purba Medinipur",
    "Chenango River (NY, USA) 11.9": "Chenango River, NY USA",
    "Chenango River (NY, USA) 7.0": "Chenango River, NY USA",
    "Eğirdir Lake, Turkey": "Egirdir Lake, Turkey",
    "Çaltı Brook": "Calti Brook",
    "Garças Lagoon": "Garcas Lagoon",
    "Shari–Goyain / Shari Goyain River (Bangladesh)": "Shari Goyain River, Bangladesh",
    "Nuʻuanu Reservoir (Oʻahu, Hawaii)": "Nuʻuanu Reservoir, Oʻahu, Hawaii",
    "Lake Worth Drainage Canal L-14": "Lake Worth Drainage Canal L14",
    "Lake Eber (Akarçay basin)": "Lake Eber, Akarcay basin",
    "Lake Beyşehir": "Lake Beysehir",
    "Lake Akşehir": "Lake Aksehir",
    "Mekong Delta — brackish estuaries (Vietnam)": "Mekong Delta, Vietnam",
    "Moreton Bay — mangrove creeks (tidal)": "Moreton Bay",
    "Jubail / northern Arabian Gulf (Saudi coast) — coastal waters where specimen recorded": "Jubail Northern Arabian Gulf",
    "Chalakudy River, Puliyarkutti 8th bridge": "Chalakudy River, Puliyarkutti 8th bridge",
    "Danube, Garla Mare (S1, Romania)": "Danube, Garla Mare",
    "Danube, Bogdan Secian (S3)": "Danube, Bogdan Secian",
    "Danube, Belene (S7)": "Danube, Belene",
    "Danube, Vardim (S8)": "Danube, Vardim",
    "Danube, Iantra (S9)": "Danube, Iantra",    
    "Danube, Kosui (S11)": "Danube, Kosui",
    "Trini rivers: Oropuche, Ortoire, Cuesa, Guapo": "Trini rivers",
    "Saskatchewan – Nelson River populations (Canada)": "Saskatchewan Nelson River",
    "Pacific populations: Fraser, North Thompson, Similkameen Rivers (BC, Canada)": "PSimilkameen Rivers BC, Canada",
    "Akköprü Stream": "Akkopru Stream",
    "Lake Väddöviken, Sweden": "Lake Vaddoviken, Sweden",
    "Lake Eğirdir, Turkey": "Lake Egirdir, Turkey",
    "Sacramento–San Joaquin Delta": "Sacramento San Joaquin Delta",
    "Lake Buir / Buir Lake (Mongolia / China border; sometimes spelled Buyr/Buir)": "Buir Lake, Mongolia",
    "Muar River floodplain (Malaysia) – sluggish/turbid ditches & floodplain": "Muar River floodplain, Malaysia",
    "Southern Thailand – rice paddies/swamps/ditches": "Southern Thailand",
    "Aceh peat swamps (Indonesia) – among plant roots ~50–70 cm deep": "Aceh peat swamps, Indonesia",
    "Nuʻuanu Reservoir, Oʻahu, Hawaii": "Nuʻuanu Reservoir, Oʻahu, Hawaii",
    "Lake Worth Drainage Canal L-14": "Lake Worth Drainage Canal",
    "Lake Eber (Akarçay basin)": "Lake Eber, Akarcay basin",
    "Lake Beyşehir": "Lake Beysehir",
    "Lake Akşehir": "Lake Aksehir",
    "Chalakudy River, Puliyarkutti 8th bridge": "Chalakudy River, Puliyarkutti",
    "Lake Dąbie, Poland": "Lake Dabie, Poland",
    "Pond in India (2025 study)": "India Pond",
    "River Soča, Slovenia": "River Soca, Slovenia",
    "Laguna de Bay (Station 2)": "Laguna de Bay",
    "Laguna de Bay (Station 3)": "Laguna de Bay",
    "Manurigao River (Station 2)": "Manurigao River",
    "Manurigao River (Station 3)": "Manurigao River",
    "Karataş Lake, Turkey": "Karatas Lake, Turkey",
    "Bistrița-Năsăud, Romania (trout farms)": "Bistrita Nasaud, Romania",
    "Soča River (Slovenia)": "Soca River, Slovenia",
    "Čelik Lake, Adıyaman, Turkey": "Celik Lake, Adiyaman, Turkey",
    "Eğlenhoca Reservoir, Turkey": "Eglenhoca Reservoir, Turkey",
    "Florida (USA) – introduced (Palm Beach & Hillsborough counties)": "Florida, USA",
    "Cueva del Azufre (surface creek 'El Azufre')": "Cueva del Azufre",
    "General range: shallow, warm springs, margins of streams/rivers": "General range",
    "Tampa Bay ditches/reservoirs, Oʻahu (e.g., Lake Wilson)": "Tampa Bay ditches reservoirs, Oʻahu",
    "Coastal rivers of WAfrica (Casamance → St John)": "Coastal rivers of WAfrica",
    "Atlantic-slope streams, S. Tamaulipas–N. Veracruz, Mexico": "Atlantic slope streams, Mexico",
    "Hainan Island (first wild record, China) — site values reported in invasion record": "Hainan Island, China",
    "C-51 Canal (Palm Beach County), Florida, USA": "C51 Canal, Florida, USA",
    "Bandama River estuary, Azagny (Côte d’Ivoire)": "Bandama River estuary, Azagny",
    "Mill Creek at FM 149, USA": "Mill Creek, USA",
    "Nuʻuanu Reservoir, Oʻahu, Hawaii": "Nuʻuanu Reservoir, Oʻahu, Hawaii",
    "Lake Worth Drainage Canal L-14": "Lake Worth Drainage Canal",
    "Lake Eber (Akarçay basin)": "Lake Eber, Akarcay basin",
    "Lake Beyşehir": "Lake Beysehir",
    "Lake Akşehir": "Lake Aksehir",
    "Cueva del Azufre (surface creek 'El Azufre')": "Cueva del Azufre",
    "Tampa Bay ditches reservoirs, Oʻahu": "Tampa Bay ditches reservoirs, Oʻahu",
    "Southern Texas ditch/lagoon sites (WES, LPK — Heubel field sites)": "Southern Texas lagoon sites",
    "Banc d’Arguin (Mauritania)": "Banc d'Arguin, Mauritania",
    "Ciénaga de Capote (UEP1)": "Cienaga de Capote",
    "Canal del Dique / Compuerticas (UEP2)": "Canal del Dique",
    "Ciénaga del Jobo / San Cristóbal (UEP4)": "Cienaga del Jobo",
    "Ciénaga del Totumo (UEP9)": "Cienaga del Totumo",
    "Potomac River drainage, Maryland (non–native reported)": "Potomac River drainage, Maryland",
    "Ngongo–Noumbi rivers": "Ngongo Noumbi rivers",
    "Tributary 4 (Waterfall Creek)": "Waterfall Creek",
    "Roanoke River at Salem USGS 02054550 snapshot 2025-09-03 08:55 EDT": "Roanoke River, Salem",
    "Maputo Bay, Mozambique (coastal/estuarine sites; aggregated monitoring)": "Maputo Bay, Mozambique",
    "Pearl Harbor, Oahu, Hawaii (introduced record; museum / NAS occurrence)": "Pearl Harbor, Oahu, Hawaii",
    "Caloosahatchee River at S-79 (Caloosahatchee estuary outlet), Florida, USA": "Caloosahatchee River, Florida, USA",
    "Puerto Rico rivers and reservoirs (Dos Bocas Lake; Loíza River; Lucchetti Reservoir)": "Puerto Rico rivers and reservoirs",
    "Razim Lake / Razelm (Romania, Razim–Sinoe lacustrine complex)": "Razim Lake, Romania",
    "Danube River pre-delta (Braila–Isaccea monitoring region, Romania)": "Danube River, Romania",
    "Snapper Creek Canal (C-2), Miami area (USA)": "Snapper Creek Canal, Miami area",
    "Berg River — Wellington station (example site)": "Berg River, Wellington",
    "Olifants (Clanwilliam) system — Clanwilliam Dam area": "Olifants Clanwilliam system",
    "Choapa River basin (central Chile: Choapa, Quilimari etc.)": "Choapa River basin, Chile",
    "Mogi-Guaçu floodplain lake, Brazil": "Mogi Guacu floodplain lake, Brazil",
    "Nuʻuanu Reservoir, Oʻahu, Hawaii": "Nuʻuanu Reservoir, Oʻahu, Hawaii",
    "Lake Worth Drainage Canal L-14": "Lake Worth Drainage Canal",
    "Lake Eber (Akarçay basin)": "Lake Eber, Akarcay basin",
    "Lake Beyşehir": "Lake Beysehir",
    "Lake Akşehir": "Lake Aksehir",
    "Cueva del Azufre (surface creek \"El Azufre\")": "Cueva del Azufre",
    "Tampa Bay ditches reservoirs, Oʻahu": "Tampa Bay ditches reservoirs, Oʻahu",
    "Lake Tanganyika — Bujumbura / northern basin (Burundi)": "Lake Tanganyika, Bujumbura",
    "Lake Tanganyika — Mpulungu / southern basin (Zambia)": "Lake Tanganyika, Mpulungu",
    "Lake Tanganyika — Kalemie / eastern basin (DRC)": "Lake Tanganyika, Kalemie",
    "Lake Tanganyika — whole-lake representative (surface epilimnion)": "Lake Tanganyika",
    "Singapore reservoirs (representative: MacRitchie / Lower Peirce)": "Singapore reservoirs",
    "C51 Canal, Florida, USA": "C51 Canal, Florida",
    "Don River lower reaches (Russia; invaded via Volga–Don canal)": "Don River, Russia",
    "Danube River — JDS4 site JDS4-3 (surface water sample 09.07.2019)": "Danube River",
    "Danube River — JDS4 site JDS4-21 (surface water sample 01.07.2019)": "Danube River",
    "Danube River (littoral average across JDS campaigns — summer)": "Danube River",
    "Thousand Islands / upper St. Lawrence River (recent detections, 2016 onward)": "Lawrence River",
    "Caloosahatchee River at S-79 (estuary/river), USA": "Caloosahatchee River, Florida",
    "Stones River — King Pond — sample 2004-10-06, USA": "Stones River, King Pond",
    "Stones River — King Pond — sample 2006-08-02, USA": "Stones River, King Pond",
    "Stones River — Redoubt Brannan, USA": "Stones River, Redoubt Brannan",
    "Caloosahatchee River estuary (S 79)": "Caloosahatchee River, Florida",
    "Lake 223, Canada": "Lake 223, Canada",
    "New River — New River at Thurmond": "New River, Thurmond",
    "Laurel Fork (SR 1111), Watauga River basin, NC": "Laurel Fork, Watauga River",
    "Dutch Creek (SR 1112 / NC 194), Watauga basin, NC": "Dutch Creek, Watauga basin",
    "Cove Creek (off SR 1121), Watauga basin, NC": "Cove Creek, Watauga basin",
    "Nuʻuanu Reservoir, Oʻahu, Hawaii": "Nuʻuanu Reservoir, Oʻahu",
    "Lake Worth Drainage Canal L-14 ": "Lake Worth Drainage Canal",
    "Lake Eber (Akarçay basin) ": "Lake Eber, Akarcay basin",
    "Lake Beyşehir ": "Lake Beysehir",
    "Lake Akşehir ": "Lake Aksehir",
    "Tampa Bay ditches reservoirs, Oʻahu": "Tampa Bay ditches reservoirs, Oʻahu",
    "C51 Canal, Florida": "C51 Canal, Florida",
    "Lake 223, Canada": "Lake 223, Canada",
    "Santa Ana River (Rialto → Prado Basin), USA": "Santa Ana River",
    "Santa Ana River (sites with suckers, Rialto → Mission Inn Ave.), USA": "Santa Ana River",
    "Santa Ana River Reach 3 (Below Prado Dam), USA": "Santa Ana River",
    "Brandvlei Dam (Greater Brandvlei) — Breede system": "Brandvlei Dam",
    "Berg River — Bien Donne station (example site)": "Berg River",
    "Trinidad: Caroni River tributaries and quiet backwaters": "Trinidad, Caroni River",
    "Soldier Meadows Spring #1, USA": "Soldier Meadows Spring",
    "Soldier Meadows Spring #2, USA": "Soldier Meadows Spring",
    "Sacramento–San Joaquin Delta, USA": "Sacramento San Joaquin Delta, USA",
    "C-111 Canal, USA": "C111 Canal, USA",
    "Ruby Marsh–Ruby Lake, USA": "Ruby Marsh Ruby Lake, USA",
    "Lake Eğirdir, Türkiye": "Lake Egirdir, Turkey",
    "Lake Beyşehir, Türkiye": "Lake Beysehir, Turkey",
    "Kızılırmak River, Türkiye": "Kizilirmak River, Turkey",
    "Upper Rio Tietê Drainage, Brazil": "Upper Rio Tiete Drainage, Brazil",
    "Đồng Tháp Floodplain, Vietnam": "Dong Thap Floodplain, Vietnam",
    "Kampar River tributary — Station": "Kampar River",
    "Upper Danube — Austrian / German reach": "Upper Danube, Austria Germany",
    "Jia Bharali River — Site S1 (Dharikati)": "Jia Bharali River, Dharikati",
    "Jia Bharali River — Site S2 (Potasali)": "Jia Bharali River, Potasali",
    "Mānoa Stream (Woodlawn Drive area), Oʻahu, Hawai‘i": "Mānoa Stream, Woodlawn Drive area, Oʻahu, Hawai‘i",
    "Pea River (PR1)": "Pea River",
    "Pea River (PR2)": "Pea River",
    "Pea River (P3)": "Pea RiveR",
    "Susquehanna River — SUSQ 231.7": "Susquehanna River",
    "Susquehanna River — SUSQ 96.3": "Susquehanna River",
    "Laurel Creek (SR 1123), Watauga basin, NC": "Laurel Creek, Watauga basin",
    "Cranberry Creek (NC 194), Watauga basin, NC": "Cranberry Creek, Watauga basin",
    "Choctawhatchee River (CR-1, AL)": "Choctawhatchee River, AL",
    "Pea River (PR-3, AL)": "Pea River, AL",
    "Yellow River (YR-1, AL)": "Yellow River, AL",
    "Halawakee Creek @ HACL-1 (Russell County, Alabama)": "Halawakee Creek, Russell County",
    "Tangipahoa River watershed — Tributary station 1": "Tangipahoa River watershed",
    "Tangipahoa River watershed — Tributary station 3": "Tangipahoa River watershed",
    "Tangipahoa River watershed — Tributary station 6": "Tangipahoa River watershed",
    "Little River Ditch no. 1 near Morehouse, MO": "Little River Ditch, Morehouse, MO",
    "Black Cypress Bayou near U.S. Highway 59": "Black Cypress Bayou",
    "Little Cypress Bayou near U.S. Highway 59": "Little Cypress Bayou",
    "Seto Inland Sea — Hinase": "Seto Inland Sea, Hinase",
    "Mimbres River — at Mimbres / near Deming ": "Mimbres River, Deming",
    "Pearl River (MS — Jackson / downstream monitoring)": "Pearl River, MS",
    "Lake Wānaka ": "Lake Wanaka",
    "Rio Grande — Embudo (Taos area / high-elevation reach)": "Rio Grande, Embudo Taos",
    "Mānoa Stream, Woodlawn Drive area, Oʻahu, Hawai‘i": "MAnoa Stream",
    "Ariake Sea — inner Ariake": "Ariake Sea",
    "Ise Bay — inner basin": "Ise Bay",
    "Sefīd-Rūd River system": "Sefid Rud River system",
    "Jia Bharali River — Site S3 (Kalabil)": "Jia Bharali River, Kalabil",
    "Jia Bharali River — Site S4 (Garami Loga)": "Jia Bharali River, Garami Loga",
    "Jia Bharali River — Site S5 (Bhalukpung)": "Jia Bharali River, Bhalukpung",
    "Ayeyarwady (Irrawaddy) River — Mandalay region": "Ayeyarwady River, Mandalay",
    "Sittaung (Sittang) River — Bago region": "Sittaung River, Bagon",
    "Lake Neuchâtel (Lac de Neuchâtel) — Switzerland": "Lake Neuchatel, Switzerland",
    "Barguzin River (Russia — eastern Baikal tributary)": "Barguzin River, Russia",
    "Kichera River (Baikal tributary — western/southern catchment small tributary)": "Kichera River, Baikal tributary",
    "Wadi Hesu’a": "Wadi Hesua",
    "Ain Al-Hammah–Khaibar": "Ain Al Hammah Khaibar",
    "Qusaiba’a Dam": "Qusaibaa Dam",
    "Lake Victoria (Site 3)": "Lake Victoria",
    "Shari–Goyain River (Bangladesh)": "Shari Goyain River, Bangladesh",
    "Água das Araras Stream": "Agua das Araras Stream",
    "Água Boa Stream": "Agua Boa Stream",
    "Bear River — at Evanston, WY": "Bear River, Evanston",
    "Smiths Fork — at Cokeville, WY ": "Smiths Fork, Cokeville",
    "Pacific Creek — at Moran, WY ": "Pacific Creek, Moran",
    "Gros Ventre River — at Kelly / NPS boundary ": "Gros Ventre River, Kelly",
    " Nuʻuanu Reservoir, Oʻahu": "Nuʻuanu Reservoir, Oʻahu",
    "Tampa Bay ditches reservoirs, Oʻahu": "Tampa Bay, Oʻahu",
    "C51 Canal, Florida": "C51 Canal, Florida",
    "Lake 223, Canada": "Lake 223, Canada",
    "C111 Canal, USA": "C111 Canal, USA",
    "Rio Grande — Embudo": "Rio Grande, Embudo",
    "Rio Grande — Otowi Bridge": "Rio Grande, Otowi Bridge",
    "Rio Chama — near Chamita": "Rio Chama",
    "Pecos River — below Brantley/Carlsbad": "Pecos River",
    "Rio Grande — Otowi Bridge (near San Ildefonso / Santa Fe reach)": "Rio Grande, Otowi Bridge",
    "Rio Chama — Chamita / upper Rio Chama tributary sites": "Rio Chama, Chamita",
    "Alamosa River / San Luis Valley tributaries (CO — headwater streams & Swale Lake area)": "Alamosa River",
    "Pecos River — (upper tributaries / headwater reaches in NM)": "Pecos River",
    "Virgin River — at Virgin, UT": "Virgin River",
    "Virgin River — at Littlefield, AZ": "Virgin River",
    "Virgin River — near St. George, UT": "Virgin River",
    "Virgin River — below La Verkin Hot Springs / La Verkin reach": "Virgin River",
    "Muddy River — near Moapa/Overton, NV": "Muddy River, Moapa Overton",
    "Bogotá River — upper basin": "Bogota River",
    "Laguna de Fúquene (Ubaté–Chiquinquirá)": "Laguna de Fuquene",
    "Etowah River — Allatoona/above Cartersville (GA)": "Etowah River, Allatoona",
    "Coosa River — Mayos Ba": "Coosa River, Mayos Ba",
    "Dan River at NC 704 near Francisco": "Dan River",
    "North Platte River — lower/near North Platte, NE": "North Platte RiverE",
    "Três Marias Reservoir (SFR upstream)": "Tres Marias , SFR",
    "Três Marias Reservoir": "Tres Marias Reservoir",
    "Ayamaru Lake Tributary 1": "Ayamaru Lake",
    "Ayamaru Lake Tributary 2": "Ayamaru Lake",
    "Rio Grande — Embudo ": "Rio Grande, Embudo",
    "Pecos River — below Brantley/Carlsbad ": "Pecos River",
    "Virgin River — at Littlefield, AZ ": "Virgin River",
    "Muddy River — near Moapa/Overton, NV ": "Muddy River, Moapa Overton",
    "Bogotá River — upper basin ": "Bogota River",
    "Laguna de Fúquene (Ubaté–Chiquinquirá) ": "Laguna de Fuquene",
    "North Platte River — lower/near North Platte, NE ": "North Platte River",
    "Sacramento River — Freeport / Deep Water Ship Channel ": "Sacramento River, Freeport",
    "San Joaquin River — Vernalis ": "San Joaquin River, Vernalis",
    "Russian River — Hopland / Guerneville reaches ": "Russian River, Hopland",
    "Buffalo Fork — near Moran ": "Buffalo Fork, Moran",
    "Rock Creek drainage — Fossil Butte / Kemmerer area ": "Rock Creek",
    "Mississippi River — St. Louis ": "Mississippi River, St. Louis",
    "Cache River — at mouth near Clarendon, AR": "Cache River, Clarendon",
    "Big Muddy / LaRue–Pine Hills/backwater  ": "Big Muddy, LaRue Pine Hills",
    "Flint River — at Albany, GA": "Flint River, Albany",
    "Flint River — near Thomaston / mid-Flint": "Flint River, Thomaston",
    "Chattahoochee River — above Lake Lanier / Buford Dam area ": "Chattahoochee ",
    "Chipola River — at Marianna / Chipola shoal reaches ": "Chipola River, Marianna",
    "Volta River — Kpong Reach": "Volta River, Kpong",
    "Mahaweli River — Gampola Reach": "Mahaweli River, Gampola",
    "Mahaweli River — Polgolla Reach": "Mahaweli River, Polgolla",
    "Doring River — Rietkuil Reach": "Doring River, Rietkuil",
    "Big Muddy / LaRue–Pine Hills/backwater lakes ": "Big Muddy, LaRue Pine Hills",
    "Apalachicola River — below Jim Woodruff / Apalachicola shoal zone": "Apalachicola River, Jim Woodruff",
    "Logan Martin Reservoir / Coosa River — ": "Coosa River",
    "Cahaba River — near West Blocton ": "Cahaba River, West Blocton",
    "Alabama River — at Montgomery": "Alabama River, Montgomery",
    "Lower Yangtze / connected floodplain lakes (China — representative lower Yangtze reaches)": "Lower Yangtze lakes",
    "Cape Fear River at Lock 1 near Kelly, NC ": "Cape Fear River",
    "Chattahoochee River at Columbus / 14th St": "Chattahoochee River, Columbus",
    "Black River — Black Rock / Pocahontas reach (Lawrence / Randolph Co., AR)": "Black River",
    "Red River — tailwaters / Lake Texoma (Denison Dam / immediate tailwater, TX/OK)": "Red River, Lake Texoma",
    "Spring River / Spring River basin (Arkansas records & basin studies; representative stations)": "Spring River",
    "Dale’s Gorge Pool": "Dale Gorge Pool",
    "Çarşamba Stream": "Carsamba Stream",
    "Águeda River": "Águeda River",
    "Lake Eğirdir": "Lake Egirdir",
    "Lake Gölcük": "Lake Golcuk",
    "O’Donnell Canyon AZ": "O’Donnell Canyon, AZ",
    "Rio Grande – Isleta Reach": "Rio Grande, Isleta",
    "Rio Grande – Angostura Reach": "Rio Grande, Angostura",
    "Rio Grande - San Acacia Reach": "Rio Grande, San Acacia",
    "Rio Grande - Cochiti Reach": "Rio Grande, Cochiti",
    "Volta River - Senchi Reach": "Volta River, Senchi",
    "Volta River - Akuse Reach": "Volta River, Akuse",
    "Volta River - Kpong Reach": "Volta River, Kpong",
    "Mahaweli River – Gampola Reach": "Mahaweli River, Gampola",
    "Mahaweli River – Polgolla Reach": "Mahaweli River, Polgolla",
    "Doring River – Rietkuil Reach": "Doring River, Rietkuil",
    "Olifants River – Clanwilliam Reach": "Olifants River, Clanwilliam",
    "Biedouw River – Upper Tributary": "Biedouw River, Upper",
    "Little Colorado River – Upper Reach": "Little Colorado River, Upper",
    "Bensbach River – Lower Reach": "Bensbach River, Lower",
    "Bensbach River – Mid Reach": "Bensbach River, Mid",
    "Bensbach River – Upper Reach": "Bensbach River, Upper",
    "Mubi Creek – Limestone Reach": "Mubi Creek, Limestone",
    "Kikori River – Tributary 7": "Kikori River",
    "River Swat – Panjkora confluence (Bosaq)": "River Swat, Panjkora",
    "Near Mangoro area (mine-region rivers: Torotorofotsy, Sakalava, etc.)": "Mangoro Rivers",
    "Lake Tsimanampesotse (SW Madagascar) — different habitat, but actual values": "Lake Tsimanampesotse",
    "Lake Alaotra (wetlands) — more general lake data": "Lake Alaotra",
    "Rio Grande – San Acacia Reach": "Rio Grande, San Acasia",
    "Rio Grande – Cochiti Reach": "Rio Grande, Cochiti",
    "Volta River – Senchi Reach": "Volta River, Senchi",
    "Volta River – Akuse Reach": "Volta River, Akuse",
    "Volta River – Kpong Reach": "Volta River, Kpong",
    "Mubi Creek – Forested Headwaters": "Mubi Creek",
    "St. Lawrence River – Lavaltrie Reach": "St. Lawrence River",
    "Shannon River – Headwaters": "Shannon River",
    "Lake Buško Blato": "Lake Busko",
    "Eğirdir Lake": "Egirdir Lake",
    "Korana–Struga Tributary": "Korana Struga Tributary",
    "Lake Akgöl": "Lake Akgol",
    "Zamantı Stream": "Zamanti Stream",
    "Karagöl Tributary": "Karagol",
    "Körkün Stream": "Korkun Stream",
    "Lake Taal – Eastern Bay": "Lake Taal, Eastern Bay",
    "Lake Taal – Southern Basin": "Lake Taal, Southern Basin",
    "Lake Taal – Volcano Island West": "Lake Taal, Volcano Island West",
    "Lake Matano – Salonsa Beach": "Lake Matano, Salonsa Beach",
    "Lake Matano – Petea River Inlet": "Lake Matano, Petea Inlet",
    "Lake Matano – Central Basin": "Lake Matano, Central Basin",
    "Tinana–Coondoo Creek": "Tinana Coondoo Creek",
    "Île Sainte-Marie Stream": "Ile Sainte Marie Stream",
    "Sarıöz Stream": "Sarioz Stream",
    "Hatchery tanks (Sarma et al. 2025)": "Hatchery Tanks",
    "Lab surgery setup (Majhi et al. 2020)": "Lab Surgery Setup",
    "Mid‐Snowy River system (VIC/NSW)": "Snowy River System",
    "Lake Lindu (Lore Lindu NP, Central Sulawesi). FishBase+1": "Lake Lindu",
    "Mwanza Gulf (Kenyan/Tanzanian sector of Lake Victoria) — selected fishing beaches / sampling sites used in the cited study. SCIRP": "Mwanza Gulf, Lake Victoria",
    "Lake Acıgöl": "Lake Acigol",
    "Lower Murray–Darling wetlands": "Lower Murray Darling Wetlands",
    "River Swat–Panjkora confluence (Bosaq)": "River Swat, Panjkora",
    "Cauvery River – Karur stretch (S1–S8)": "Cauvery River, Karur",
    "Cauvery River – Pallipalayam stretch": "Cauvery River, Pallipalayam",
    "Río Cana (Dec 2018)": "Río Cana",
    "Río Cana (May 2019)": "Río Cana",
    "Fasin River area (general West Papua) – aquarium hobby data": "Fasin River, West Papua",
    "Bool Lagoon & area between Beachport–Robe, SA": "Bool Lagoon",
    "Elmalı Stream (İznik Basin)": "Elmali Stream, Iznik Basin",
    "Alakır Stream": "Alakir Stream",
    "Tahtalı River, Turkey": "Tahtali River, Turkey",
    "Lake Buško Blato, Bosnia & Herzegovina": "Lake Busko",
    "Lake Prološko Blato, Croatia": "Lake Prolosko Blato, Croatia",
    "Sušik Creek": "Susik Creek",
    "Jasenčica Creek": "Jasencica Creek",
    "Lake Prošće": "Lake Prosće",
    "Yıldızeli River": "Yildizeli River",
    "Taşköprü River": "Taskopru River",
    "Missouri River, Montana–Dakota (USA)": "Missouri River",
    "Loukkos River estuary — stations S1–S5 near Larache, Morocco": "Loukkos River",
    "Devils River — Pafford Crossing (Val Verde Co.), Texas, US": "Devils River",
    "Kerala backwaters / rivers (type locality: Cochin area; SW India)": "Kerala backwatersa",
    "Lake Prosće": "Lake Prosce",
    "Suşehri River": "Susehri River",
    "Pınarbaşı River": "Pinarbasi River",
    "Örenşehir River": "Orensehir River",
    "Akdağmadeni River": "Akdagmadeni River",
    "Arroyo de Ávalos, Jalisco": "Arroyo de Avalos, Jalisco",
    "Stajnička Jaruga, Croatia": "Stajnicka Jaruga, Croatia",
    "Čikola River, Croatia": "Cikola River, Croatia",
    "Buško Blato Lake, Bosnia": "Busko Blato Lake, Bosnia",
    "Lake Beyşehir, Turkey": "Lake Beysehir, Turkey",
    "Çayköy Stream, Turkey": "Caykoy Stream, Turkey",
    "Lake Fegen (Sjön Fegen), Sweden": "Lake Fegen, Sweden",
    "Lake Poso, Station S1 (Koburo)": "Lake Poso, Koburo",
    "Lake Poso, Station S2 (Bo'e)": "Lake Poso, Boe",
    "Lake Poso, Station S3 (Tolambo)": "Lake Poso, Tolambo",
    "Lake Poso, Station S4 (Peura)": "Lake Poso, Peura",
    "Lake Poso, Station S5 (Tentena)": "Lake Poso, Tentena",
    "Seridó (upper Piranhas–Açu basin), Rio Grande do Norte, NE Brazil": "Serido, Rio Grande do Norte",
    "Curaçao (ABC islands)": "Curacao",
    "São Francisco River basin — Rio das Velhas subcatchment (Minas Gerais, Brazil)": "Sao Francisco River basin",
    "Lake Victoria — Mwanza Gulf": "Mwanza Gulf, Lake Victoria",
    "Rio Iguaçu and coastal drainages of Rio Grande do Sul & Santa Catarina (Brazil)": "Rio Iguacu Coastal Drainages Brazil",
    "Hann River — Moll Gorge, WA Australia": "Hann River ",
    "Sulawesi & nearby Indonesian islands — upland clear streams, waterfalls": "Sulawesi Streams",
    "Lake Chapala / Lerma–Chapala Basin (western-central Mexico)": "Lake Chapala Basin Mexico",
    "Rio Grande, Texas–New Mexico (USA)": "Rio Grande Texas New Mexico USA",
    "Upper Fly River basin & small tributaries (Fly R., PNG) — representative: Fly River tributaries near Kiunga": "Upper Fly River Basin",
    "Intermountain / Bonneville & Snake basins; streams & rivers in western USA (native range).": "Bonneville and Snake Basins",
    "São Francisco River basin (Brazil) — calm/slow sections.": "Sao Francisco River Basin",
    "Pacific & Atlantic versant rivers of Panama (small rivers; Gorgona island region)": "Panama Versant Rivers",
    "Lorentz / upper Fly–Strickland rivers (PNG/Indonesia)": "Lorentz Upper Fly Strickland Rivers",
    "West/Central African rivers (Senegal–Chad, Nile)": "Senegal Chad Nile",
    "Central & eastern Panama — Pacific & Atlantic slopes": "Panama Rivers",
    "Cuba — ponds, lakes, clear sluggish streams": "Cuba Ponds",
    "Cuba — ponds, lakes, streams; clear stagnant or sluggish water": "Cuba Ponds",
    "Cuba — pools and highland streams": "Cuba Highland Streams",
    "Asia & Oceania — Philippines, New Guinea, Solomon Is.; freshwater creeks/streams": "Asia and Oceania Freshwater Streams",
    "South / Southeast Asian hill streams (genus Garra common in Western Ghats and SE Asia) — species records in India region. ResearchGateVlaams Instituut voor de Zee": "South and Southeast Asian Hill Streams",
    "Coastal rivers of India (Tamil Nadu → Goa). FishBaseFishkeeper": "India Tamil Nadu Goa River",
    "East Asian freshwater bodies (China / SE Asia distribution — bitterlings use slow moving waters with unionid bivalves). FishBase": "Asia Bitterlings",
    "Streams/rivers in South/Southeast Asia (Nemacheilus spp. distributed across Asian streams) — species-specific locality data sparse online": "South and Southeast Asian Streams",
    "Papua New Guinea — upper Fly River system (Kiunga & Ningerum vicinity)": "PNG Upper Fly River",
    "France — upper Loire, Dordogne, Lot and Tarn drainages (hilly streams)": "Upper Loire Dordogne",
    "Central/Eastern Europe — Tisza / Danube drainages (small rivers & streams)": "Tisza Danube, Europe",
    "Zeravshan River (Central Asia) — Uzbekistan": "Zeravshan River, Uzbekistan",
    "Middle–upper Rio São Francisco basin": "Upper Sao Francisco River",
    "Papua New Guinea — brackish mangrove estuaries, tidal creeks, lowermost stream reaches": "PNG Mangrove Estuaries",
    "Grijalva–Usumacinta basin (Mexico)": "Grijalva Usumacinta Basin Mexico",
    "Lake Beyşehir (Beyşehir Gölü), Turkey (type locality / main freshwater record)": "Lake Beysehir, Turkey",
    "Rio das Velhas / São Francisco basin (Brazil — type locality area, Lagoa Santa region)": "Rio das Velhas, Brazil",
    "Lake Skadar (Shkodra) basin & Morača River (Montenegro)": "Lake Skadar Moraca, Montenegro"
})


                waterbody_name
75    Nuʻuanu Reservoir, Oʻahu
1770          Tampa Bay, Oʻahu
2205        C51 Canal, Florida
2344          Lake 223, Canada
2630           C111 Canal, USA
4205              Águeda River
4278      O’Donnell Canyon, AZ


# Check columns with ranges
split ranges into min max 

In [123]:
# columns that contain ranges
range_cols = [
    "temp_max",
    "weight_max", 
    "length_max", 
    "temp_pref_min",
    "temp_pref_max", 
    "temp_range", 
    "fecundity_mean", 
    "fecundity_min",
    "fecundity_max", 
    "trophic_lvl_estimate", 
    "trophic_lvl",
    "wb_ph_min",
    "wb_ph_max",
    "wb_salinity_min",
    "wb_salinity_max",
    "wb_do_min",
    "wb_do_max",
    "wb_bod_min",
    "wb_bod_max",
    "wb_turbidity_min",
    "wb_turbidity_max",
    "wb_temp_min"
    "wb_temp_max",
]

import re

def has_range(val):
    if pd.isna(val):
        return False
    val = str(val)
    # match patterns like "5-10" or "7 to 12"
    return bool(re.search(r"\d+\s*[-–—]\s*\d+|\d+\s*to\s*\d+", val))

range_cols_auto = [
    col for col in df.columns 
    if df[col].astype(str).apply(has_range).any()
]

print("Detected range columns:", range_cols_auto)

Detected range columns: ['feeding_type', 'length_max', 'temp_range', 'trophic_lvl_estimate']


In [ ]:
print(df.columns)

# df.drop(["length_max_min", "length_max_max"], axis=1, inplace=True)

Index(['fish_id', 'species', 'common_name', 'waterbody_name', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'status', 'feeding_type',
       'temp_max', 'weight_max', 'length_max', 'temp_pref_min',
       'temp_pref_max', 'temp_range', 'fecundity_mean', 'fecundity_min',
       'fecundity_max', 'trophic_lvl_estimate', 'trophic_lvl', 'wb_ph_min',
       'wb_ph_max', 'wb_salinity_min', 'wb_salinity_max', 'wb_do_min',
       'wb_do_max', 'wb_bod_min', 'wb_bod_max', 'wb_turbidity_min',
       'wb_turbidity_max', 'wb_temp_min', 'wb_temp_max', 'temp_range_min',
       'temp_range_max', 'trophic_lvl_estimate_min',
       'trophic_lvl_estimate_max'],
      dtype='object')


# Clean Feeding type

In [197]:
df['feeding_type'] = df['feeding_type'].str.strip().str.lower()

In [227]:
# Collapse into core feeding types
df.loc[df['feeding_type'].str.contains("carni", case=False, na=False), 'feeding_type'] = "carnivore"
df.loc[df['feeding_type'].str.contains("omniv", case=False, na=False), 'feeding_type'] = "omnivore"
df.loc[df['feeding_type'].str.contains("pred", case=False, na=False), 'feeding_type'] = "predator"
df.loc[df['feeding_type'].str.contains("inver", case=False, na=False), 'feeding_type'] = "invertivore"
df.loc[df['feeding_type'].str.contains("detri", case=False, na=False), 'feeding_type'] = "detritivore"
df.loc[df['feeding_type'].str.contains("grazer|graze|gra", case=False, na=False), 'feeding_type'] = "grazer"
df.loc[df['feeding_type'].str.contains("insect", case=False, na=False), 'feeding_type'] = "insectivore"
df.loc[df['feeding_type'].str.contains("herb", case=False, na=False), 'feeding_type'] = "herbivore"
df.loc[df['feeding_type'].str.contains("plankt", case=False, na=False), 'feeding_type'] = "planktivore"
df.loc[df['feeding_type'].str.contains("browser", case=False, na=False), 'feeding_type'] = "browser"
df.loc[df['feeding_type'].str.contains("scaveng", case=False, na=False), 'feeding_type'] = "scavenger"
df.loc[df['feeding_type'].str.contains("parasite", case=False, na=False), 'feeding_type'] = "parasite"
df.loc[df['feeding_type'].str.contains("pisciv", case=False, na=False), 'feeding_type'] = "carnivore"

# ✅ Now force everything else into "other"
valid_labels = [
    "carnivore","omnivore","predator","invertivore","detritivore",
    "grazer","insectivore","herbivore","planktivore","browser",
    "scavenger","parasite"
]
df.loc[~df['feeding_type'].isin(valid_labels), 'feeding_type'] = "other"

# Check result
print(df['feeding_type'].value_counts())


feeding_type
other          2277
predator       1565
omnivore        807
grazer          226
carnivore       212
insectivore     200
planktivore     108
browser          65
herbivore        45
invertivore      43
detritivore      27
scavenger         7
parasite          1
Name: count, dtype: int64


In [231]:
df.to_csv("til_feeding_type.csv", index=False)

# Clean Status

In [230]:
pd.set_option("display.max_rows", None)  # show all
print(df['status'].unique())

['Established' 'Reported' 'Native' 'Native / Established' nan 'Invasive'
 'Recorded' 'Failed' 'Extirpated' 'Present' 'Introduced' 'Endemic'
 'Abundant' 'Endangered' 'Rare' 'Extinct' 'Reintroduced' 'Translocated'
 'Declining' 'Omnivore' 'EN' 'CR' 'extinct' 'endemic' 'possibly extinct'
 'Critically Endangered / Possibly Extinct' 'native / possibly extinct'
 'failed' 'established' 'Native/endemic' 'Piscivorous/Carnivore'
 'Native/Endemic' 'Possibly Extinct' 'Native / established'
 'Reported / endemic' 'Endemic/Possibly Extinct' 'Critically Endangered'
 'LC' 'Native/Reported' 'Native/established' 'Reported/Native'
 'Native, established' 'Native/Established' 'Native/Amphidromous'
 'Endemic/Restricted' 'Native/Localized' 'Name/Established'
 'Native/Marine' 'Native/Local Endemic' 'Native/Widespread'
 'Native/Riverine' 'Native/Regional' 'Threatened/Native'
 "Native, endemic'" 'Native, frequent littoral' 'Endangered, native'
 'Endemic/Established' 'Endemic/Native' ' Endemic' 'Native, threatened

In [232]:
def clean_status(val):
    if pd.isna(val):
        return "other"
    
    val = val.strip().lower()
    
    # Priority rules (order matters!)
    if "fail" in val:
        return "failed"
    if "inva" in val:
        return "invasive"
    if "extir" in val:
        return "extirpated"
    if "est" in val:
        return "established"
    if "report" in val:
        return "reported"
    
    # Conservation status
    if val in ["en", "endangered"]:
        return "EN"
    if val in ["cr", "critically endangered"]:
        return "CR"
    if val in ["lc", "least concern"]:
        return "LC"
    if "threat" in val:
        return "Locally Threatened"
    
    return "other"

df['status'] = df['status'].apply(clean_status)

In [234]:
print(df['status'].value_counts())


status
established           1456
invasive              1410
reported               800
EN                     787
other                  405
failed                 381
CR                     249
LC                      50
extirpated              39
Locally Threatened       6
Name: count, dtype: int64


In [236]:
df.to_csv("til_status.csv", index=False)

# Reorder columns
reorder columns, drop temp_range and trophic level est (already split to min max)

In [235]:
print(df.columns)

Index(['fish_id', 'species', 'common_name', 'waterbody_name', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'status', 'feeding_type',
       'temp_max', 'weight_max', 'length_max', 'temp_pref_min',
       'temp_pref_max', 'temp_range', 'fecundity_mean', 'fecundity_min',
       'fecundity_max', 'trophic_lvl_estimate', 'trophic_lvl', 'wb_ph_min',
       'wb_ph_max', 'wb_salinity_min', 'wb_salinity_max', 'wb_do_min',
       'wb_do_max', 'wb_bod_min', 'wb_bod_max', 'wb_turbidity_min',
       'wb_turbidity_max', 'wb_temp_min', 'wb_temp_max', 'temp_range_min',
       'temp_range_max', 'trophic_lvl_estimate_min',
       'trophic_lvl_estimate_max'],
      dtype='object')


In [237]:
df = df.drop(columns=['trophic_lvl_estimate', 'temp_range'])
print(df.columns)

Index(['fish_id', 'species', 'common_name', 'waterbody_name', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'status', 'feeding_type',
       'temp_max', 'weight_max', 'length_max', 'temp_pref_min',
       'temp_pref_max', 'fecundity_mean', 'fecundity_min', 'fecundity_max',
       'trophic_lvl', 'wb_ph_min', 'wb_ph_max', 'wb_salinity_min',
       'wb_salinity_max', 'wb_do_min', 'wb_do_max', 'wb_bod_min', 'wb_bod_max',
       'wb_turbidity_min', 'wb_turbidity_max', 'wb_temp_min', 'wb_temp_max',
       'temp_range_min', 'temp_range_max', 'trophic_lvl_estimate_min',
       'trophic_lvl_estimate_max'],
      dtype='object')


In [239]:
# reorder columns

df = df[['fish_id', 'species', 'common_name', 
       'kingdom','phylum', 'class', 'order', 'family', 'genus', 
       'status', 'feeding_type',
       'temp_max', 'weight_max', 'length_max', 'temp_pref_min', 'temp_pref_max', 'temp_range_min',
       'temp_range_max', 'trophic_lvl_estimate_min','trophic_lvl_estimate_max', 'trophic_lvl', 
       'fecundity_mean', 'fecundity_min', 'fecundity_max',  
       'waterbody_name', 'wb_ph_min', 'wb_ph_max', 
       'wb_salinity_min', 'wb_salinity_max', 
       'wb_do_min', 'wb_do_max', 
       'wb_bod_min', 'wb_bod_max', 
       'wb_turbidity_min', 'wb_turbidity_max', 
       'wb_temp_min', 'wb_temp_max']]

print(df.columns)

Index(['fish_id', 'species', 'common_name', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'status', 'feeding_type', 'temp_max',
       'weight_max', 'length_max', 'temp_pref_min', 'temp_pref_max',
       'temp_range_min', 'temp_range_max', 'trophic_lvl_estimate_min',
       'trophic_lvl_estimate_max', 'trophic_lvl', 'fecundity_mean',
       'fecundity_min', 'fecundity_max', 'waterbody_name', 'wb_ph_min',
       'wb_ph_max', 'wb_salinity_min', 'wb_salinity_max', 'wb_do_min',
       'wb_do_max', 'wb_bod_min', 'wb_bod_max', 'wb_turbidity_min',
       'wb_turbidity_max', 'wb_temp_min', 'wb_temp_max'],
      dtype='object')


# CSV

1. rename columns 
2. split min max values of water params
3. fix waterbody names that uses special letters
4. split trophic lvl est and temp range to min max columns since columns consists of ranges
5. standardized feeding type to 13 valid labels
6. standardized status to 10 valid labels

all dropped columns from raw dataset:
sources, unnamed col, raw water params columns, trophic lvl est, temp range

In [240]:
df.to_csv("final_dataset_v1.csv", index=False)